# SomaTrack: Predicting Study-Related Physical Ailments

## Project Overview
Prolonged sedentary behavior in universities leads to specific physical health issues — primarily **back pain**, **neck strain**, and **tension headaches**. SomaTrack is a Machine Learning project that predicts the likelihood of experiencing these physical ailments based on students' study environment and habits.

**Problem Type**: Multiclass Classification

**Target Labels**:
- 0: No Pain
- 1: Mild / Occasional Pain  
- 2: Moderate / Frequent Pain
- 3: Severe / Chronic Pain

---

## Table of Contents
1. [Import Libraries](#1-import-libraries)
2. [Load Dataset](#2-load-dataset)
3. [Data Overview & Understanding](#3-data-overview--understanding)
4. [Data Cleaning](#4-data-cleaning)
5. [Feature Engineering](#5-feature-engineering)
6. [Exploratory Data Analysis (EDA)](#6-exploratory-data-analysis-eda)
7. [Feature Selection & Importance](#7-feature-selection--importance)
8. [Data Preprocessing for Modeling](#8-data-preprocessing-for-modeling)
9. [Model Development & Comparison](#9-model-development--comparison)
10. [Model Evaluation & Validation](#10-model-evaluation--validation)
11. [Final Model Selection & Explainability](#11-final-model-selection--explainability)
12. [Save Model & Demo](#12-save-model--demo)
13. [Conclusions & Recommendations](#13-conclusions--recommendations)

---
## 1. Import Libraries

In [ ]:
# Core
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Evaluation
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    f1_score, precision_score, recall_score, ConfusionMatrixDisplay
)

# Feature Importance
from sklearn.inspection import permutation_importance

# Model Persistence
import joblib

# Settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print('All libraries imported successfully!')

---
## 2. Load Dataset

In [ ]:
# Load the survey data
# Update the path once you have the CSV from Google Forms
df = pd.read_csv('../data/raw/somatrack_survey.csv')

# Make a copy to preserve the original
df_original = df.copy()

print(f'Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns')
df.head()

---
## 3. Data Overview & Understanding

In [ ]:
# Basic info
print('='*60)
print('DATASET INFO')
print('='*60)
df.info()
print()

print('='*60)
print('STATISTICAL SUMMARY (Numerical Features)')
print('='*60)
df.describe()

In [ ]:
# Check for missing values
print('='*60)
print('MISSING VALUES')
print('='*60)
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Percentage (%)': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if missing_df.empty:
    print('No missing values found!')
else:
    print(missing_df)

print()
print('='*60)
print('DUPLICATE ROWS')
print('='*60)
print(f'Number of duplicate rows: {df.duplicated().sum()}')

In [ ]:
# Check data types and unique values for each column
print('='*60)
print('COLUMN DETAILS')
print('='*60)
for col in df.columns:
    print(f'\n--- {col} ---')
    print(f'  dtype: {df[col].dtype}')
    print(f'  unique values: {df[col].nunique()}')
    if df[col].nunique() <= 10:
        print(f'  value counts:')
        print(df[col].value_counts().to_string())

---
## 4. Data Cleaning

Tasks:
- Handle missing values
- Remove duplicates
- Fix data types
- Rename columns for consistency
- Handle outliers

In [ ]:
# ============================================================
# RENAME COLUMNS
# ============================================================
# Rename Google Forms long column names to short, clean names
# UPDATE this mapping based on your actual Google Form questions

column_mapping = {
    # 'Original Google Form Question': 'clean_name'
    # Example mappings (update with your actual column names):
    # 'What is your age?': 'age',
    # 'What is your gender?': 'gender',
    # 'How many hours do you spend sitting per day while studying?': 'sitting_hours_per_day',
    # 'How many consecutive hours do you study without taking a break?': 'consecutive_hours_no_break',
    # 'How many liters of water do you drink per day?': 'water_liters_per_day',
    # 'What is your total daily screen time (phone + laptop) in hours?': 'daily_screen_time_hours',
    # 'Where do you primarily study?': 'study_location',
    # 'What type of seat do you usually use while studying?': 'seat_type',
    # 'Do you use external peripherals (mouse/keyboard)?': 'uses_peripherals',
    # 'How often do you exercise per week?': 'exercise_frequency',
    # 'Do you take regular breaks during study sessions?': 'takes_breaks',
    # 'How would you rate your overall posture while studying? (1-10)': 'posture_rating',
    # 'Rate your back pain level (0-3)': 'back_pain_level',
    # 'Rate your neck pain level (0-3)': 'neck_pain_level',
    # 'Rate your headache frequency (0-3)': 'headache_level',
    # 'Overall pain level (0-3)': 'pain_level',
}

# df.rename(columns=column_mapping, inplace=True)  # Uncomment after filling mapping
print('Columns after renaming:')
print(df.columns.tolist())

In [ ]:
# ============================================================
# DROP UNNECESSARY COLUMNS
# ============================================================
# Drop the Google Forms timestamp column and any other irrelevant columns

columns_to_drop = ['Timestamp']  # Add any other columns to drop
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

print(f'Shape after dropping columns: {df.shape}')

In [ ]:
# ============================================================
# HANDLE MISSING VALUES
# ============================================================

# Strategy:
# - Numerical columns: fill with median (robust to outliers)
# - Categorical columns: fill with mode (most frequent value)

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f'Numerical columns ({len(numerical_cols)}): {numerical_cols}')
print(f'Categorical columns ({len(categorical_cols)}): {categorical_cols}')

# Fill missing values
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)
        print(f'  Filled {col} with median: {df[col].median()}')

for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)
        print(f'  Filled {col} with mode: {df[col].mode()[0]}')

print(f'\nRemaining missing values: {df.isnull().sum().sum()}')

In [ ]:
# ============================================================
# REMOVE DUPLICATES
# ============================================================
before = len(df)
df.drop_duplicates(inplace=True)
after = len(df)
print(f'Removed {before - after} duplicate rows. Remaining: {after} rows.')

In [ ]:
# ============================================================
# DETECT & HANDLE OUTLIERS (using IQR method)
# ============================================================

def detect_outliers_iqr(dataframe, column):
    """Detect outliers using IQR method."""
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = dataframe[(dataframe[column] < lower) | (dataframe[column] > upper)]
    return outliers, lower, upper

print('Outlier Detection (IQR Method):')
print('-' * 50)
for col in numerical_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    if len(outliers) > 0:
        print(f'{col}: {len(outliers)} outliers (range: {lower:.2f} - {upper:.2f})')

# Optional: cap outliers instead of removing them
# for col in numerical_cols:
#     Q1 = df[col].quantile(0.25)
#     Q3 = df[col].quantile(0.75)
#     IQR = Q3 - Q1
#     df[col] = df[col].clip(Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)

In [ ]:
# Save cleaned data
df.to_csv('../data/processed/somatrack_cleaned.csv', index=False)
print(f'Cleaned dataset saved: {df.shape[0]} rows, {df.shape[1]} columns')

---
## 5. Feature Engineering

Create new features that may improve model performance.

In [ ]:
# ============================================================
# CREATE NEW FEATURES
# ============================================================

# Example engineered features (adjust column names based on your actual data):

# 1. Break ratio: how often breaks are taken relative to study time
# df['break_ratio'] = df['consecutive_hours_no_break'] / df['sitting_hours_per_day']

# 2. Sedentary intensity: screen time * sitting hours
# df['sedentary_intensity'] = df['daily_screen_time_hours'] * df['sitting_hours_per_day']

# 3. Hydration per sitting hour
# df['hydration_per_hour'] = df['water_liters_per_day'] / df['sitting_hours_per_day']

# 4. Is the study environment ergonomic? (binary)
# df['ergonomic_setup'] = ((df['seat_type'] == 'Ergonomic chair') & 
#                          (df['uses_peripherals'] == 'Yes')).astype(int)

# 5. Risk score: combination of bad habits
# df['risk_score'] = (df['sitting_hours_per_day'] * 0.3 + 
#                     df['consecutive_hours_no_break'] * 0.3 + 
#                     df['daily_screen_time_hours'] * 0.2 - 
#                     df['water_liters_per_day'] * 0.1 - 
#                     df['exercise_frequency'] * 0.1)

print('Feature engineering complete.')
print(f'Dataset shape: {df.shape}')
df.head()

---
## 6. Exploratory Data Analysis (EDA)

Comprehensive visualizations to understand our data and campus ergonomic health.

### 6.1 Target Variable Distribution

In [ ]:
# ============================================================
# TARGET VARIABLE DISTRIBUTION
# ============================================================
# UPDATE 'pain_level' to your actual target column name

TARGET = 'pain_level'  # <-- UPDATE THIS

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
pain_counts = df[TARGET].value_counts().sort_index()
colors = ['#2ecc71', '#f39c12', '#e74c3c', '#8e44ad']
pain_labels = ['No Pain', 'Mild', 'Moderate', 'Severe']

axes[0].bar(pain_counts.index, pain_counts.values, color=colors[:len(pain_counts)])
axes[0].set_xlabel('Pain Level')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Pain Levels')
axes[0].set_xticks(pain_counts.index)
axes[0].set_xticklabels(pain_labels[:len(pain_counts)])

# Pie chart
axes[1].pie(pain_counts.values, labels=pain_labels[:len(pain_counts)], 
            autopct='%1.1f%%', colors=colors[:len(pain_counts)], startangle=90)
axes[1].set_title('Pain Level Proportions')

plt.tight_layout()
plt.show()

print('\nClass Distribution:')
print(df[TARGET].value_counts().sort_index())

### 6.2 Numerical Features Distribution

In [ ]:
# ============================================================
# NUMERICAL FEATURES - Histograms & Boxplots
# ============================================================

num_features = [col for col in numerical_cols if col != TARGET]
n = len(num_features)

fig, axes = plt.subplots(n, 2, figsize=(14, 4 * n))
if n == 1:
    axes = axes.reshape(1, -1)

for i, col in enumerate(num_features):
    # Histogram
    axes[i, 0].hist(df[col], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    axes[i, 0].set_title(f'{col} - Distribution')
    axes[i, 0].set_xlabel(col)
    axes[i, 0].set_ylabel('Frequency')
    
    # Boxplot
    axes[i, 1].boxplot(df[col].dropna(), vert=True)
    axes[i, 1].set_title(f'{col} - Box Plot')
    axes[i, 1].set_ylabel(col)

plt.tight_layout()
plt.show()

### 6.3 Categorical Features Distribution

In [ ]:
# ============================================================
# CATEGORICAL FEATURES - Count plots
# ============================================================

cat_features = [col for col in categorical_cols if col != TARGET]
n_cat = len(cat_features)

if n_cat > 0:
    fig, axes = plt.subplots(1, n_cat, figsize=(6 * n_cat, 5))
    if n_cat == 1:
        axes = [axes]
    
    for i, col in enumerate(cat_features):
        sns.countplot(data=df, x=col, ax=axes[i], palette='viridis')
        axes[i].set_title(f'{col} Distribution')
        axes[i].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print('No categorical features to plot.')

### 6.4 Pain Levels Across Study Locations

In [ ]:
# ============================================================
# PAIN LEVELS vs STUDY LOCATION
# ============================================================
# UPDATE column names as needed

STUDY_LOCATION = 'study_location'  # <-- UPDATE THIS

if STUDY_LOCATION in df.columns:
    fig = px.histogram(df, x=STUDY_LOCATION, color=TARGET, 
                       barmode='group',
                       title='Pain Level Distribution Across Study Locations',
                       labels={STUDY_LOCATION: 'Study Location', TARGET: 'Pain Level'},
                       color_discrete_sequence=px.colors.qualitative.Set2)
    fig.show()
    
    # Cross-tabulation
    print('\nCross-tabulation: Study Location vs Pain Level')
    print(pd.crosstab(df[STUDY_LOCATION], df[TARGET], margins=True))
else:
    print(f'Column "{STUDY_LOCATION}" not found. Update the variable name.')

### 6.5 Pain Levels Across Seat Types

In [ ]:
# ============================================================
# PAIN LEVELS vs SEAT TYPE
# ============================================================

SEAT_TYPE = 'seat_type'  # <-- UPDATE THIS

if SEAT_TYPE in df.columns:
    fig = px.histogram(df, x=SEAT_TYPE, color=TARGET,
                       barmode='group',
                       title='Pain Level Distribution Across Seat Types',
                       labels={SEAT_TYPE: 'Seat Type', TARGET: 'Pain Level'},
                       color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.show()
else:
    print(f'Column "{SEAT_TYPE}" not found. Update the variable name.')

### 6.6 Correlation Heatmap

In [ ]:
# ============================================================
# CORRELATION HEATMAP (numerical features only)
# ============================================================

plt.figure(figsize=(14, 10))
corr_matrix = df[numerical_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.tight_layout()
plt.show()

### 6.7 Pairplot of Key Features vs Pain Level

In [ ]:
# ============================================================
# PAIRPLOT - Key features vs target
# ============================================================
# Select the most important numerical features for pairplot

# UPDATE these with your actual column names
key_features = numerical_cols[:4]  # Take first 4 numerical features, or specify manually

if len(key_features) > 1:
    plot_df = df[key_features + [TARGET]].copy()
    plot_df[TARGET] = plot_df[TARGET].astype(str)
    sns.pairplot(plot_df, hue=TARGET, palette='Set2', diag_kind='kde')
    plt.suptitle('Pairplot of Key Features by Pain Level', y=1.02)
    plt.show()
else:
    print('Not enough numerical features for pairplot.')

---
## 7. Feature Selection & Importance

Identify which study habits and environmental factors contribute most to physical pain.

In [ ]:
# ============================================================
# FEATURE IMPORTANCE using Random Forest
# ============================================================

# Prepare data for feature importance analysis
df_encoded = df.copy()

# Encode categorical variables
label_encoders = {}
for col in categorical_cols:
    if col in df_encoded.columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

# Split features and target
X_imp = df_encoded.drop(columns=[TARGET])
y_imp = df_encoded[TARGET]

# Train a Random Forest for feature importance
rf_importance = RandomForestClassifier(n_estimators=100, random_state=42)
rf_importance.fit(X_imp, y_imp)

# Get feature importances
importances = pd.DataFrame({
    'Feature': X_imp.columns,
    'Importance': rf_importance.feature_importances_
}).sort_values('Importance', ascending=True)

# Plot
fig = px.bar(importances, x='Importance', y='Feature', orientation='h',
             title='Feature Importance (Random Forest)',
             color='Importance', color_continuous_scale='RdYlGn_r')
fig.update_layout(height=500)
fig.show()

print('\nTop Risk Factors (Features most correlated with pain):')
print(importances.tail(5).to_string(index=False))

In [ ]:
# ============================================================
# PERMUTATION IMPORTANCE (more robust measure)
# ============================================================

X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(
    X_imp, y_imp, test_size=0.2, random_state=42, stratify=y_imp
)

rf_perm = RandomForestClassifier(n_estimators=100, random_state=42)
rf_perm.fit(X_train_imp, y_train_imp)

perm_imp = permutation_importance(rf_perm, X_test_imp, y_test_imp, 
                                   n_repeats=10, random_state=42)

perm_df = pd.DataFrame({
    'Feature': X_imp.columns,
    'Importance Mean': perm_imp.importances_mean,
    'Importance Std': perm_imp.importances_std
}).sort_values('Importance Mean', ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(perm_df['Feature'], perm_df['Importance Mean'], 
        xerr=perm_df['Importance Std'], color='coral')
ax.set_title('Permutation Feature Importance')
ax.set_xlabel('Mean Importance')
plt.tight_layout()
plt.show()

---
## 8. Data Preprocessing for Modeling

Encode categorical features, scale numerical features, and split data.

In [ ]:
# ============================================================
# PREPARE DATA FOR MODELING
# ============================================================

# Use the encoded dataframe
X = df_encoded.drop(columns=[TARGET])
y = df_encoded[TARGET]

print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')
print(f'\nTarget distribution:')
print(y.value_counts().sort_index())

In [ ]:
# ============================================================
# TRAIN-TEST SPLIT (Stratified)
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Training set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')
print(f'\nTraining target distribution:')
print(y_train.value_counts().sort_index())

In [ ]:
# ============================================================
# FEATURE SCALING
# ============================================================

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for readability
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print('Feature scaling complete.')
print(f'\nScaled training data sample:')
X_train_scaled.head()

---
## 9. Model Development & Comparison

Train and compare multiple classification models.

In [ ]:
# ============================================================
# DEFINE MODELS
# ============================================================

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
}

print(f'Models to train: {list(models.keys())}')

In [ ]:
# ============================================================
# TRAIN & EVALUATE ALL MODELS
# ============================================================

results = []

for name, model in models.items():
    print(f'\n{"="*60}')
    print(f'Training: {name}')
    print(f'{"="*60}')
    
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    
    results.append({
        'Model': name,
        'Accuracy': acc,
        'F1 (Weighted)': f1_weighted,
        'F1 (Macro)': f1_macro,
        'Precision': precision,
        'Recall': recall
    })
    
    print(f'Accuracy: {acc:.4f}')
    print(f'F1 (Weighted): {f1_weighted:.4f}')
    print(f'F1 (Macro): {f1_macro:.4f}')
    print(f'\nClassification Report:')
    print(classification_report(y_test, y_pred))

# Summary table
results_df = pd.DataFrame(results).sort_values('F1 (Weighted)', ascending=False)
print('\n' + '='*60)
print('MODEL COMPARISON SUMMARY')
print('='*60)
results_df

In [ ]:
# ============================================================
# VISUAL MODEL COMPARISON
# ============================================================

fig = px.bar(results_df, x='Model', y=['Accuracy', 'F1 (Weighted)', 'F1 (Macro)', 'Precision', 'Recall'],
             barmode='group', title='Model Performance Comparison',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(yaxis_title='Score', xaxis_title='Model', legend_title='Metric')
fig.show()

---
## 10. Model Evaluation & Validation

### 10.1 Confusion Matrices

In [ ]:
# ============================================================
# CONFUSION MATRICES for all models
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (name, model) in enumerate(models.items()):
    y_pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot(ax=axes[i], cmap='Blues', colorbar=False)
    axes[i].set_title(f'{name}')

plt.suptitle('Confusion Matrices - All Models', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 10.2 Stratified K-Fold Cross-Validation

In [ ]:
# ============================================================
# STRATIFIED K-FOLD CROSS-VALIDATION
# ============================================================

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = []

# Scale all data for CV
X_all_scaled = scaler.fit_transform(X)

for name, model in models.items():
    scores = cross_val_score(model, X_all_scaled, y, cv=skf, scoring='f1_weighted')
    cv_results.append({
        'Model': name,
        'CV Mean F1': scores.mean(),
        'CV Std F1': scores.std(),
        'CV Scores': scores
    })
    print(f'{name}: F1 = {scores.mean():.4f} (+/- {scores.std():.4f})')

cv_df = pd.DataFrame(cv_results)[['Model', 'CV Mean F1', 'CV Std F1']].sort_values('CV Mean F1', ascending=False)
print('\n' + '='*60)
print('CROSS-VALIDATION RESULTS')
print('='*60)
cv_df

In [ ]:
# ============================================================
# CV RESULTS BOXPLOT
# ============================================================

cv_scores_dict = {r['Model']: r['CV Scores'] for r in cv_results}
cv_box_df = pd.DataFrame(cv_scores_dict)

fig, ax = plt.subplots(figsize=(12, 6))
cv_box_df.boxplot(ax=ax)
ax.set_title('Stratified 5-Fold Cross-Validation F1 Scores')
ax.set_ylabel('F1 Score (Weighted)')
ax.set_xlabel('Model')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## 11. Final Model Selection & Explainability

Select the best model and make its decision logic explainable (White-box AI).

In [ ]:
# ============================================================
# SELECT BEST MODEL
# ============================================================

best_model_name = cv_df.iloc[0]['Model']
best_model = models[best_model_name]

# Retrain on full training data
best_model.fit(X_train_scaled, y_train)
y_pred_best = best_model.predict(X_test_scaled)

print(f'Best Model: {best_model_name}')
print(f'\nFinal Classification Report:')
print(classification_report(y_test, y_pred_best))

# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=pain_labels[:len(np.unique(y))],
            yticklabels=pain_labels[:len(np.unique(y))])
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# MODEL EXPLAINABILITY - Decision Tree Visualization
# ============================================================
# Decision Trees are inherently interpretable (White-box)

from sklearn.tree import export_text, plot_tree

# Train a simple Decision Tree for explainability
dt_explainer = DecisionTreeClassifier(max_depth=4, random_state=42)
dt_explainer.fit(X_train_scaled, y_train)

# Text representation
tree_rules = export_text(dt_explainer, feature_names=list(X.columns))
print('Decision Tree Rules (max_depth=4):')
print(tree_rules)

# Visual representation
plt.figure(figsize=(20, 10))
plot_tree(dt_explainer, feature_names=list(X.columns), 
          class_names=pain_labels[:len(np.unique(y))],
          filled=True, rounded=True, fontsize=8)
plt.title('Decision Tree Visualization (Explainable Model)')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# KEY RISK FACTORS SUMMARY
# ============================================================

print('='*60)
print('TOP 3 RISK FACTORS (Actionable Insights)')
print('='*60)

top_features = importances.tail(3)
for i, (_, row) in enumerate(top_features.iterrows(), 1):
    print(f'\n{i}. {row["Feature"]} (Importance: {row["Importance"]:.4f})')

print('\n' + '='*60)
print('RECOMMENDATIONS')
print('='*60)
print('Based on the analysis, the following changes can reduce pain risk:')
print('(These will be filled based on actual feature importance results)')

---
## 12. Save Model & Demo

Save the trained model and create an interactive demo.

In [ ]:
# ============================================================
# SAVE MODEL & SCALER
# ============================================================

joblib.dump(best_model, '../models/somatrack_model.joblib')
joblib.dump(scaler, '../models/somatrack_scaler.joblib')
joblib.dump(label_encoders, '../models/somatrack_encoders.joblib')

print(f'Model saved: somatrack_model.joblib')
print(f'Scaler saved: somatrack_scaler.joblib')
print(f'Encoders saved: somatrack_encoders.joblib')

In [ ]:
# ============================================================
# INTERACTIVE DEMO - Ergonomic Risk Audit
# ============================================================

def predict_pain_risk(input_data: dict):
    """
    Predict pain risk for a student based on their study habits.
    
    Parameters:
    -----------
    input_data : dict
        Dictionary with feature names as keys and values.
        Example: {
            'sitting_hours_per_day': 6,
            'consecutive_hours_no_break': 3,
            'water_liters_per_day': 1.5,
            'daily_screen_time_hours': 8,
            'study_location': 'Bed',
            'seat_type': 'No chair (bed)',
            'uses_peripherals': 'No',
            ...
        }
    
    Returns:
    --------
    Prediction and risk assessment.
    """
    # Load model, scaler, encoders
    model = joblib.load('../models/somatrack_model.joblib')
    scaler = joblib.load('../models/somatrack_scaler.joblib')
    encoders = joblib.load('../models/somatrack_encoders.joblib')
    
    # Create DataFrame
    input_df = pd.DataFrame([input_data])
    
    # Encode categorical features
    for col, le in encoders.items():
        if col in input_df.columns:
            input_df[col] = le.transform(input_df[col].astype(str))
    
    # Scale
    input_scaled = scaler.transform(input_df)
    
    # Predict
    prediction = model.predict(input_scaled)[0]
    
    risk_labels = {
        0: 'No Pain - Low Risk',
        1: 'Mild Pain - Moderate Risk',
        2: 'Frequent Pain - High Risk',
        3: 'Chronic Pain - Very High Risk'
    }
    
    print('='*50)
    print('SOMATRACK - ERGONOMIC RISK AUDIT')
    print('='*50)
    print(f'\nInput: {input_data}')
    print(f'\nPrediction: {risk_labels.get(prediction, "Unknown")}')
    print(f'Pain Level: {prediction}')
    print('='*50)
    
    return prediction

print('Demo function ready. Use predict_pain_risk() with a dictionary of study habits.')

In [ ]:
# ============================================================
# DEMO EXAMPLE
# ============================================================
# UPDATE feature names and values based on your actual data

# Example: A student who studies in bed for 6 hours with no mouse
sample_student = {
    # UPDATE these keys to match your actual column names
    # 'sitting_hours_per_day': 6,
    # 'consecutive_hours_no_break': 3,
    # 'water_liters_per_day': 1.0,
    # 'daily_screen_time_hours': 8,
    # 'study_location': 'Bed',
    # 'seat_type': 'No seat (bed/couch)',
    # 'uses_peripherals': 'No',
    # 'exercise_frequency': 0,
    # 'takes_breaks': 'No',
    # 'posture_rating': 3,
}

# Uncomment the line below once sample_student is filled:
# predict_pain_risk(sample_student)

print('Update the sample_student dictionary with your actual feature names and run the demo.')

---
## 13. Conclusions & Recommendations

### Key Findings
- *TODO: Summarize the top risk factors identified*
- *TODO: Which model performed best and why*
- *TODO: Distribution of pain levels across the student population*

### Actionable Recommendations
1. **Risk Factor 1**: *TODO: e.g., "Students studying in bed are X% more likely to experience neck pain"*
2. **Risk Factor 2**: *TODO: e.g., "Taking breaks every 2 hours reduces pain risk by Y%"*
3. **Risk Factor 3**: *TODO: e.g., "Using an ergonomic chair reduces back pain by Z%"*

### Model Performance
- Best Model: *TODO*
- F1 Score (Weighted): *TODO*
- The model is explainable (White-box AI) through Decision Tree visualization

### Limitations
- Self-reported data may have bias
- Sample size considerations
- Cross-sectional study (not longitudinal)

---
*SomaTrack - Predicting Study-Related Physical Ailments*